<a href="https://colab.research.google.com/github/adxpillar/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/Adewale_Adeagbo_DS13_LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [0]:
# Engineer new feature from interest level
df.groupby('interest_level').count()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
interest_level,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
high,3690,3690,3690,3650,3671,3690,3690,3690,3688,3690,3690,3690,3690,3690,3690,3690,3690,3690,3690,3690,3690,3690,3690,3690,3690,3690,3690,3690,3690,3690,3690,3690,3690
low,33946,33946,33946,32628,33859,33946,33946,33946,33940,33946,33946,33946,33946,33946,33946,33946,33946,33946,33946,33946,33946,33946,33946,33946,33946,33946,33946,33946,33946,33946,33946,33946,33946
medium,11181,11181,11181,11114,11154,11181,11181,11181,11179,11181,11181,11181,11181,11181,11181,11181,11181,11181,11181,11181,11181,11181,11181,11181,11181,11181,11181,11181,11181,11181,11181,11181,11181


In [0]:
# function to generate dummies
def int_level(cell):
  if cell == 'high':
    return 2
  elif cell == 'medium':
    return 1
  else:
    return 0

In [0]:
df['interest_level'] = df['interest_level'].apply(int_level)

In [0]:
# Second feature to check if address description is null
df['display_address'].isnull().sum()

133

In [0]:
# check if cell is empty
def display_address(cell):
  if pd.isnull(cell):
    return 0
  else:
    return 1

In [0]:
# apply function 
df['display_add_present'] = df['display_address'].apply(display_address)

In [7]:
# confirm function worked 
df['display_add_present'].unique()

array([1, 0])

In [0]:
# more confirmation 
df.groupby('display_add_present').count()

In [0]:
# split train and test data
train = df[df['created'].str.contains('2016-04') | df['created'].str.contains('2016-05')]
test = df[df['created'].str.contains('2016-06')]

In [0]:
# Linear regression 
from sklearn.linear_model import LinearRegression

In [11]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,display_add_present
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,0,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,2,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [0]:
# instantiate linear regression 
model = LinearRegression()

In [0]:
# specify features and target 
features  = ['bathrooms','doorman','fitness_center','outdoor_space','balcony']
x_train = train[features]
x_test  = test[features]

target = 'price'
y_train = train[target]
y_test = test[target]

In [0]:
# fit the model 
model.fit(x_train,y_train)
y_pred = model.predict(x_test)

In [14]:
# model intercept 
print(model.intercept_)

368.5830444038711


In [15]:
coeff_df = pd.DataFrame(model.coef_,features,columns=['Coefficient'])
coeff_df

,Coefficient
bathrooms,2437.918679
doorman,516.404261
fitness_center,156.675403
outdoor_space,0.392731
balcony,132.090514


In [17]:

from sklearn import metrics 
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))
print('R2:', metrics.r2_score(y_test,y_pred) )
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test,y_pred)))

MAE: 845.7956861166065
R2: 0.5118745273399397
RMSE: 1231.7078997912013
